In [7]:
import pandas as pd
from environments.envs import make_env, envs_list
from solvers.solvers import solve, solve_methods
import os
from tqdm import tqdm
from time import time

print(envs_list)
print(solve_methods)
env = make_env(envs_list[6])

['Ex10', 'Ex20', 'Ex30', 'Ex31', 'Ex40', 'froz_l_e', 'froz_l', 'taxi', 'rooms_1', 'rooms_2', 'rooms_3', 'rooms_4']
['mdpTB_VI', 'mdpTB_VIGS', 'mdpTB_PI', 'mdpTB_PIM']


(4, 16, 16)

In [10]:
env.R.shape

(16, 4)